In [15]:
from os import name
import pandas as pd
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer 
import re
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/mateo/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
ingredient_file_path='../Emission_computing/final_ingredients_emissions.csv'
df=pd.read_csv(ingredient_file_path,error_bad_lines=False)

/home/mateo/.pyenv/versions/3.8.12/envs/FritzWebInterface/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
b'Skipping line 280: expected 2 fields, saw 4\n'


In [6]:
import requests
import collections

api_key = "6f58acb8fe27ff8884f00b8451c098f5"

def get_recipe(dish):

    #call API
    url = f"https://api.edamam.com/api/recipes/v2?type=public&q={dish}&app_id=4bc274b4&app_key={api_key}"

    response = requests.request("GET",url)
    recipe = response.json()["hits"][0]['recipe']

    return recipe

def getingredients(dish):

    #get ingredients from the call
    recipe = get_recipe(dish)
    servings = recipe['yield']
    ingredients = recipe['ingredients']

    new_dict = collections.defaultdict(list)

    for i in ingredients:
        new_dict["ingredient"].append(i['food'].lower())
        new_dict["weight"].append(round((i['weight'] / servings), 2))
        new_dict["foodCategory"].append(i['foodCategory'])
    return new_dict

In [24]:
def match_ingredients(output_df):
    final_dict={'ingredient':[], 'emission':[],'weight':[]}
    missing_ingredients={'missing_ingredients':[]}
    
    for ingredient in output_df['ingredient']:
        
        # 1. Lemmatize 
        ingredient_words=re.split('\s+', ingredient)
        lemmatizer = WordNetLemmatizer()
        lemmatized_output = ' '.join([lemmatizer.lemmatize(w) for w in ingredient_words])
        ingredient_words=re.split('\s+', lemmatized_output)
        
        # 2. Try the whole lemmatized sentence
        if df[df['ingredient'].str.match(r'.*'+str(lemmatized_output)+'.*')== True].ingredient.values.size>0:
                final_dict['ingredient'].append(df[df['ingredient'].str.match(r'.*'+str(lemmatized_output)+'.*')== True].ingredient.values[0])
                final_dict['emission'].append(df[df['ingredient'].str.match(r'.*'+str(lemmatized_output)+'.*')== True].emissions.values[0])
                final_dict['weight'].append(output_df[output_df["ingredient"]==ingredient].weight.iloc[0])
                
#                 final_dict['value'].append(output_df[output_df["ingredient"]==ingredient].value.iloc[0])
#                 final_dict['metric'].append(output_df[output_df["ingredient"]==ingredient].metric.iloc[0])
        # False
        
        # 3. If the lemmatized output does not work, try the words
        else:
            for word in ingredient_words:
                # "ground" 
                # "beef"
                try:
                    final_dict['ingredient'].append(df[df['ingredient'].str.match(r'.*'+str(word)+'.*')== True].ingredient.iloc[0])
                    final_dict['emission'].append(df[df['ingredient'].str.match(r'.*'+str(word)+'.*')== True].emissions.iloc[0])
                    final_dict['weight'].append(output_df[output_df["ingredient"]==ingredient].weight.iloc[0])

#                     final_dict['value'].append(output_df[output_df["ingredient"]==ingredient].value.iloc[0])
#                     final_dict['metric'].append(output_df[output_df["ingredient"]==ingredient].metric.iloc[0])                    
                except IndexError:
                    missing_ingredients['missing_ingredients'].append(ingredient)
            
#     final_df=pd.DataFrame.from_dict(final_dict)
#     # Adding the columns "value" and metric from output_dict
#     final_df=final_df.merge(output_df,on="ingredient")
    final_df=pd.DataFrame(final_dict)

    return final_df,missing_ingredients

In [25]:
output_dict=getingredients('hamburger')

In [26]:
output_df=pd.DataFrame(output_dict)

In [27]:
output_df

,ingredient,weight,foodCategory
0,ground beef,56.70,meats
1,salt,0.64,Condiments and sauces
2,black pepper,0.32,Condiments and sauces
3,hamburger bun,21.00,"bread, rolls and tortillas"
4,tomato,16.88,vegetables
5,iceberg,2.50,vegetables
6,onion,10.40,vegetables


In [28]:
final_df,missing_ing=match_ingredients(output_df)

In [29]:
final_df

,ingredient,emission,weight
0,groundnut oil,NaN,56.70
1,beef,43330.0,56.70
2,salt,1133.0,0.64
3,black pepper,1133.0,0.32
4,hamburger bun,1215.0,21.00
5,tomatoe,2905.0,16.88
6,iceberg,2063.0,2.50
7,onion,1051.0,10.40
